In [35]:
import anndata
from collections import defaultdict
import copy
import csv
from joblib import Parallel, delayed
from matplotlib import colors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import style
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing
import networkx as nx
import numba
import numpy as np
import numpy.random as rnd
import os
import pandas as pd
import pickle
import random
from random import choices
import re
import scipy as scp
import scipy.integrate as integrate
from scipy.special import hyp2f1 as hyper
import scipy.stats as stats
from scipy.stats import norm as normal
import scvelo as scv
from scvelo.tools.velocity_embedding import quiver_autoscale,velocity_embedding
import seaborn as sns
from sklearn import preprocessing
from sklearn.cluster import Birch
import sklearn.decomposition as skd
from sklearn.neighbors import NearestNeighbors
import string
import umap
import skbio as sk
scv.settings.verbosity = 0

## Functions

### Utilities

In [17]:
def load_adata(file):
    
    # INPUT
    # file = path to file containing a pickle object
    # OUTPUT
    # AnnData object
    
    with open(file, 'rb') as inF:
        obj = pickle.load(inF)
        
        return(obj)
    
def save_adata(obj, filename):
    
    # IPUT
    # obj = python object
    # filename = path to save object

    with open(filename, 'wb') as output: 
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
def unique(list1): 
    
    # INPUT
    # list1 = python list
    # OUTPUT:
    # numpy array with unique elements in the list
    
    x = np.array(list1) 
    return(np.unique(x))

### Distances

In [33]:
def rescale(df):
    
    # INPUT
    # df =  distance matrix data frame
    # OUTPUT
    # scaled_df = scaled distance matrix
    
    scaled_df = df
    values = []
    for i in range(0,len(df.index)):
        for j in range(i,len(df.columns)):
            values.append(df.iloc[i,j])

    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(np.array(values).reshape(-1, 1))

    k=0
    for i in range(0,len(df.index)):
        for j in range(i,len(df.columns)):
            scaled_df.iloc[i,j]=x_scaled[k]
            scaled_df.iloc[j,i]=x_scaled[k]
            k=k+1
    return(scaled_df)

def expression_distance(adata,resc=True,copy=False):
    
    # INPUT
    # adata - AnnData object
    # clustcol - name of the column used to cluster cells 
    # resc - whether to normalize and rescale distances (recommended)
    # copy - whether a copy of the distance matrix should be returned. By default copy=True and adata.uns['expression_distances'] is updated

    clusters = [c for c in adata.obs.dropna()[clustcol].unique() if c!='nan']
    centroids = [np.array(np.mean(adata.layers['spliced'][(adata.obs[clustcol] == c).values,:],axis=0)[:,].tolist()[0]) for c in clusters]


    nc = len(centroids)
    dist = pd.DataFrame(0,index=range(0,nc),columns=range(0,nc),dtype=np.float64)
    for i in range(0,nc):
        for j in range(i,nc):
            dist.at[i,j] = np.linalg.norm(centroids[i] - centroids[j])
            dist.at[j,i] = dist.at[i,j]
            
    if(resc):
        dist = rescale(dist)
        
    dist = sk.DistanceMatrix(dist.values)
    
    if copy==False:
        adata.uns['expression_distances'] = dist
    else:
        return(dist)

def preprocess(adata):
    
    # Compute cluster distances 
    
    expression_distance(adata)
    
    # Remove genes with NaN's in velocity
        
    V = adata.layers["velocity"]
    genes_valid = adata.var[['velocity_genes']].iloc[np.where(np.logical_not(np.isnan(V.sum(axis=0))))[0].tolist(),:].index
    adata_valid = adata[:,genes_valid]
    adata_valid.var['gtype'] = 'Velocity_not_nan'
    
    return(adata)

### Network inference

In [ ]:
def rank_genes(V,X,n,m):
    

    # INPUT
    #   - X = velocity matric for the cells in a specific cluster
    #   - xm = expression matrix for the cells in a specific cluster
    #   - ng = number of genes to choose
    #   - m = str ranking method. Options:
    #        * abstop = gene ranking based on the mean (across cells) absolute value of velocity 
    #        * ran = gene set selected at random
    #        * top = gene ranking based on the mean (across cells) value of velocity (including sign)
    #        * minstd = gene ranking based on increasing standard deviation of velocity across cells
    #        * maxstd = gene ranking based on decreasing standard deviation of velocity across cells
    #        * vrank = gene ranking based on differential velocity across clusters
    #        * mark = cluster marker genes, specified in the object clusterMarkers, this options requires argument 'c' to be the name of the cluster
    #        * leastvar = gene ranking basedon increasing standard deviation of expression across cells
    #        * topvar = gene ranking basedon decreasing standard deviation of expression across cells
    # Returns:
    #   - genes = list with genes selected (based on the cluster matrix dimensions) 
    
    ng = int(ng)
    
    if(mode=='abstop'):
        pos = v.dropna().abs().mean(0).sort_values(ascending=False)[0:ng].index.tolist()
    elif(mode=='ran'):
        n = len(v.dropna().columns)
        if(ng>=n):
            pos = v.dropna().columns.tolist()
        else:
            pos = v.dropna().sample(ng,axis=1).abs().mean(0).index.tolist()
    elif(mode=='top'):
        pos = v.dropna().mean(0).sort_values(ascending=False)[0:ng].index.tolist()
    elif(mode=='maxstd'):
        pos = v.dropna().std(0).sort_values(ascending=False).index.tolist()[0:ng]
    elif(mode == 'vrank'):
        pos = V_rank[c][0:ng].tolist()
    elif(mode=='topvar'):        
        pos = xm.dropna().std(0).sort_values(ascending=False).index.tolist()[0:ng]
    elif(mode=='highexp'):
        pos = xm.dropna().mean(0).sort_values(ascending=False).index.tolist()[0:ng]

    return(pos)




def predict_network(adata,cluster,genes,network_size,copy=False):

    ## INPUT 
    
    # clustcol - cluster column used
    # cluster - individual cluster label
    # genes - either a str (vrank,maxstd,topvar,abstop,random) or a list of genes (same as in adata.var.index) 
    # network_size - number of genes used to infer the network
    # copy - whether a copy of the network should be returned. By default copy=True
    
    ## OUTPUT
    
    ind = adata.obs[clustcol] == cluster
    
    #Velocity matrix
    V = pd.dataFrame(adata.layers["velocity"][ind.values,:],columns=adata.var.index)
    
    #Expression matrix
    X = pd.dataFrame(adata.layers['spliced'][ind.values,:].todense(),columns=adata.var.index)

    #Get requested genes
    
    
    

    #Filter matrices for model reconstruction 
    
    genes = [g for g in fg if g in X_c.columns]
    X_c_f = X_c.loc[:,genes] # cell by gene expression matrix
    V_c_f = V_c.loc[:,genes] # cell by gene velocity matrix
    G_f = np.diag(adata.var.fit_gamma[V_non_nan].loc[genes,]) # gene by gene diagonal gamma matrix

    X_d = pd.DataFrame(np.array(X_c_f,dtype=np.float64),columns = genes)
    V_c_f = V_c_f.fillna(0)
    
    #W.index,W.columns = fg,fg

### Gene selection

Functions to run testing of user-specified combinations of gene sets and network sizes. Networks are not stored at this step, only the list of the top `n` combinations (according to the mantel correlation will be stored in adata.uns as `top_gene_sets`.

In [ ]:
def select_network_mode(adata,clustcol,genes,network_size):
    
    
    clusts = [c for c in adata.obs.dropna()[clustcol].unique() if c!='nan']
    W_list = [predict_network(adata,clustcol,c,genes,network_size,copy=True) for c in clusts]
    
    # Calculate mantel correlation between distance matrices
    net_dist = cdm_computeDistance(W_list,resc=True,dis_type='euclidean')
    coeff,p_value,_ = sk.stats.distance.mantel(net_dist,adata.uns['cluster_centroid_distances'])
    
    results = [dset_name,method,ngenes,exp_tf,coeff,p_value]
    
    return(results)

### Network perturbations

## Pipeline

In [39]:
# Input parameters

global dset,clustcol
dset = 'hFB18'
clustcol = 'labels'

In [40]:
# Load adata

adata_dir = "/Users/larisamorales/Documents/KAUST/scgrn-project/objects/"
adata_path = adata_dir + "scvelo/" + dset + "-adata.pkl"
adata = load_adata(adata_path)

In [41]:
adata = preprocess(adata)

Trying to set attribute `.var` of view, copying.
